<font size="6"><b>XGBoost - with DEP_DELAY</b></font>

![Figure_8](img/Figure_8.png)

In [1]:
import pandas as pd
import numpy as np
np.random.seed(0)
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import xgboost as xgb
sns.set_style('darkgrid')
pd.set_option('display.max_columns', None)
import datetime, warnings, scipy
warnings.filterwarnings("ignore")

from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn import svm
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report

In [2]:
dfm_ready = pd.read_csv('dfm_ready.csv', index_col=0)
dfm_ready.head()

,DEP_DELAY,CRS_ELAPSED_TIME,AIR_TIME,DISTANCE,FLIGHT_STATUS,OP_CARRIER_Allegiant Air,OP_CARRIER_American Airlines,OP_CARRIER_Delta Airlines,OP_CARRIER_Endeavor Air,OP_CARRIER_Envoy Air,OP_CARRIER_ExpressJet,OP_CARRIER_Frontier Airlines,OP_CARRIER_Hawaiian Airlines,OP_CARRIER_JetBlue Airways,OP_CARRIER_Mesa Airline,OP_CARRIER_PSA Airlines,OP_CARRIER_Republic Airways,OP_CARRIER_SkyWest Airlines,OP_CARRIER_Southwest Airlines,OP_CARRIER_Spirit Airlines,OP_CARRIER_United Airlines,OP_CARRIER_Virgin America,DEST_Atlanta,DEST_Boston,DEST_Charlotte,DEST_Chicago,DEST_Dallas-Fort Worth,DEST_Denver,DEST_Detroit,DEST_Houston,DEST_Las Vegas,DEST_Los Angeles,DEST_Minneapolis,DEST_New York,DEST_Newark,DEST_Orlando,DEST_Philadelphia,DEST_Phoenix,DEST_Salt Lake City,DEST_San Francisco,DEST_Seattle,CRS_DEP_TIME_2,CRS_DEP_TIME_3,CRS_DEP_TIME_4,CRS_ARR_TIME_2,CRS_ARR_TIME_3,CRS_ARR_TIME_4,MONTH_2,MONTH_3,MONTH_4,MONTH_5,MONTH_6,MONTH_7,MONTH_8,MONTH_9,MONTH_10,MONTH_11,MONTH_12,WEEKDAY_1,WEEKDAY_2,WEEKDAY_3,WEEKDAY_4,WEEKDAY_5,WEEKDAY_6
0,-5.0,268.0,225.0,1605.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,-8.0,99.0,65.0,414.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,-5.0,134.0,106.0,846.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,6.0,190.0,157.0,1120.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,-3.0,206.0,173.0,1222.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


As you can see in the dataframe above I have only considered features that you are aware of before you take off. This way what I am predicting is before you board the plane and not while you are in the plane in the air, which wouldn't be of much use as you would want to know if you will be late before you board the plane. Adding any of the features listed below would increase your accuracy to at least 80%, which sounds great, but then again, what's the point if you are already in the air?<br>

* TAXI_OUT
* WHEELS_OFF
* WHEELS_ON
* TAXI_IN
* ARR_DELAY
* ACTUAL_ELAPSED_TIME

Now, the first column on that dataframe is the DEP_DELAY (Departure Delay), which yes, if your plane is leaving late then your chances of arriving late to your destination will increase. I did do as part of the EDA a plot that you can see below, where I compare the DEP_DELAY with the ARR_DELAY by airline, and as you can see, normally when your flight leaves late, the airlines push for the flights to have shorter elapse times to compensate for the delay, and in some cases, this is accounted for and you arrive either on time, or even earlier, such as with Delta Airlines and Alaska airlines, which have both negative arrival averages, meaning an early arrival. 

![Figure_7](img/Figure_7.png)

Some people might argue, that if your flight's departure is delayed, you will see it on the screens before you board the plane, so that means that I should leave it on my predictive model right? well yes and no. Yes I should leave because you are right about seeing the flight's departure being delayed before you board the plane, but then no, because a late departure will most probably mean a late arrival even when the airlines try to compensate by reducing their elapsed time as the plot above suggests. So this will definitely affect the accuracy of my predictions in a positive but unrealistic predictive way. So what do I do? do I go for the yes and get a high accuracy that will impress you? or should I go for a proper predictive model with a lower accuracy that most probably won't impress you? it was hard to decide so as a solution I have done my models with and without that feature, and that way I can make a good comparison of how it will affect all the algorithm's that I'm using for my predictions. 

In this notebook I am testing XGBoost with the DEP_DELAY being considered as a predictive feature

In [12]:
dfm_ready.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4008257 entries, 0 to 7213445
Data columns (total 64 columns):
 #   Column                         Dtype  
---  ------                         -----  
 0   DEP_DELAY                      float64
 1   CRS_ELAPSED_TIME               float64
 2   AIR_TIME                       float64
 3   DISTANCE                       float64
 4   FLIGHT_STATUS                  int64  
 5   OP_CARRIER_Allegiant Air       int64  
 6   OP_CARRIER_American Airlines   int64  
 7   OP_CARRIER_Delta Airlines      int64  
 8   OP_CARRIER_Endeavor Air        int64  
 9   OP_CARRIER_Envoy Air           int64  
 10  OP_CARRIER_ExpressJet          int64  
 11  OP_CARRIER_Frontier Airlines   int64  
 12  OP_CARRIER_Hawaiian Airlines   int64  
 13  OP_CARRIER_JetBlue Airways     int64  
 14  OP_CARRIER_Mesa Airline        int64  
 15  OP_CARRIER_PSA Airlines        int64  
 16  OP_CARRIER_Republic Airways    int64  
 17  OP_CARRIER_SkyWest Airlines    int64  
 18  OP

As the info() shows, the categorical have all been dealt with so all that needs to be done is start running the model

# XGBoost

In [3]:
# Define features (X) and traget(y)
y = dfm_ready['FLIGHT_STATUS']
X = dfm_ready.drop(['FLIGHT_STATUS'], axis = 1)

In [8]:
# Standard Scaler creation using the fit_transform() method
scaler = StandardScaler()
scaled_df = scaler.fit_transform(X)

In [14]:
# Perform the dataset split
X_test, X_train, y_test, y_train = train_test_split(scaled_df, y, test_size=0.25, random_state=42)

In [15]:
clf = xgb.XGBClassifier()
clf.fit(X_train, y_train)

training_preds = clf.predict(X_train)
val_preds = clf.predict(X_test)

training_accuracy = accuracy_score(y_train, training_preds)
val_accuracy = accuracy_score(y_test, val_preds)

print("Training Accuracy: {:.4}%".format(training_accuracy * 100))
print("Validation accuracy: {:.4}%".format(val_accuracy * 100))

Training Accuracy: 82.91%
Validation accuracy: 82.83%


The fact that the test set (validation) has an accuracy so close to the training one, suggests that the model is properly fit. If there was a high difference between then (with the test being considerably lower), then that would suggest an overfit of the training data. 

Still it is always worth to try to tune the model and see if the accuracy improves. The 82.83% obtained is quite good for flight delay prediction though, but remember, I have left the DEP_DELAY which will boost the accuracy. 

Regardless, I'm going to create a parameters dictionary and with the help of the GridSearchCV, I'll see  to if I can get a set of good parameters to improve the model's performance

# Tuning XGBoost

In [16]:
param_grid = {
    "learning_rate": [0.1],
    'max_depth': [6],
    'min_child_weight': [10],
    'subsample': [ 0.7],
    'n_estimators': [5, 30, 100],
}

In [13]:
grid_clf = GridSearchCV(clf, param_grid, scoring='accuracy', cv=None, n_jobs=1)
grid_clf.fit(scaled_df, y)

best_parameters = grid_clf.best_params_

print("Grid Search found the following optimal parameters: ")
for param_name in sorted(best_parameters.keys()):
    print("%s: %r" % (param_name, best_parameters[param_name]))

training_preds = grid_clf.predict(X_train)
val_preds = grid_clf.predict(X_test)
training_accuracy = accuracy_score(y_train, training_preds)
val_accuracy = accuracy_score(y_test, val_preds)

print("")
print("Training Accuracy: {:.4}%".format(training_accuracy * 100))
print("Validation accuracy: {:.4}%".format(val_accuracy * 100))

Grid Search found the following optimal parameters: 
learning_rate: 0.1
max_depth: 6
min_child_weight: 10
n_estimators: 100
subsample: 0.7

Training Accuracy: 85.56%
Validation accuracy: 85.55%


The model improved by 2.72%, not as much as I would have expected but at least it is an improvement which is what we are looking for with the tuning

One additional test that I could do is to account for the imbalance dataset. I know that XGBoost actually performs very well with imbalance dataset. But then there is a reason for it to have the option to account for it with the <b>scale_pos_weight</b>, so I will give it a try and see if I can beat the previous results

In [40]:
grid_clf_1 = grid_clf.predict(X_test)
grid_clf_1

array([1, 0, 0, ..., 0, 1, 1])

In [41]:
print(confusion_matrix(y_test, grid_clf_1))
print(classification_report(y_test, grid_clf_1))

[[1823163   66245]
 [ 368461  748323]]
              precision    recall  f1-score   support

           0       0.83      0.96      0.89   1889408
           1       0.92      0.67      0.77   1116784

    accuracy                           0.86   3006192
   macro avg       0.88      0.82      0.83   3006192
weighted avg       0.86      0.86      0.85   3006192



# XGBoost for imbalanced Classification

Although the XGBoost algorithm performs well for a wide range of challenging problems, it offers a large number of hyperparameters, many of which require tuning in order to get the most out of the algorithm on a given dataset.

The implementation provides a hyperparameter designed to tune the behavior of the algorithm for imbalanced classification problems; this is the scale_pos_weight hyperparameter.

By default, the scale_pos_weight hyperparameter is set to the value of 1.0 and has the effect of weighing the balance of positive examples, relative to negative examples when boosting decision trees. For an imbalanced binary classification dataset, the negative class refers to the majority class (class 0) and the positive class refers to the minority class (class 1).

scale_pos_weight = total_negative_examples/total_positive_examples

In [18]:
#model = XGBClassifier(scale_pos_weight=1.6913)

In [21]:
model_imb = xgb.XGBClassifier(learning_rate=0.1,
                            max_depth = 6, 
                            n_estimators = 100,
                            scale_pos_weight=1.69)
model_imb.fit(X_train, y_train)

XGBClassifier(max_depth=6, scale_pos_weight=1.69)

In [30]:
print("Accuracy on training set: {:.2f}".format(model_imb.score(X_train, y_train) * 100))
print("Accuracy on validation set: {:.2f}".format(model_imb.score(X_test, y_test) * 100))

Accuracy on training set: 85.70
Accuracy on validation set: 85.46


The <b>training accuracy</b> slightly improved but the <b>vallidation accuracy</b> decreased. I was hoping for a better performance of the model when balancing the classes. Maybe if I play more with the values? I will try that but for the time being, I can see no improvement, and therefore I would keep the results of the imbalance XGBoost.

I will finish to produce a confusion matrix and the classification report and then test a few changes for the scale_pos_weight.

In [27]:
xgb_predict = model_imb.predict(X_test)
xgb_predict

array([1, 0, 0, ..., 0, 1, 1])

In [28]:
print(confusion_matrix(y_test,xgb_predict))
print(classification_report(y_test,xgb_predict))

[[1738987  150421]
 [ 286652  830132]]
              precision    recall  f1-score   support

           0       0.86      0.92      0.89   1889408
           1       0.85      0.74      0.79   1116784

    accuracy                           0.85   3006192
   macro avg       0.85      0.83      0.84   3006192
weighted avg       0.85      0.85      0.85   3006192



## Testing different scale_pos_weights

<b>scale_pos_weight = 2</b>

In [31]:
model_imb_t1 = xgb.XGBClassifier(learning_rate=0.1,
                            max_depth = 6, 
                            n_estimators = 100,
                            scale_pos_weight=2)
model_imb_t1.fit(X_train, y_train)

XGBClassifier(max_depth=6, scale_pos_weight=2)

In [32]:
print("Accuracy on training set: {:.2f}".format(model_imb_t1.score(X_train, y_train) * 100))
print("Accuracy on validation set: {:.2f}".format(model_imb_t1.score(X_test, y_test) * 100))

Accuracy on training set: 85.06
Accuracy on validation set: 84.82


The scale_pos_weight of 2 didn't help to improve the model's accuracy but the drop wasn't a big thing by being less than 1%. It doesn't seem as increasing the value will help, but lets keep on trying and if it doesn't work, I'll try dropping it. 

<b>scale_pos_weight = 3</b>

In [34]:
model_imb_t2 = xgb.XGBClassifier(learning_rate=0.1,
                            max_depth = 6, 
                            n_estimators = 100,
                            scale_pos_weight=3)
model_imb_t2.fit(X_train, y_train)

XGBClassifier(max_depth=6, scale_pos_weight=3)

In [35]:
print("Accuracy on training set: {:.2f}".format(model_imb_t2.score(X_train, y_train) * 100))
print("Accuracy on validation set: {:.2f}".format(model_imb_t2.score(X_test, y_test) * 100))

Accuracy on training set: 82.00
Accuracy on validation set: 81.75


Alright, so it seems as the higher I'm making the scale_pos_weeight, the more the accuaries are decreasing, so let's see what happens if I slightly decrease it

<b>scale_pos_weight = 1.3</b>

In [36]:
model_imb_t3 = xgb.XGBClassifier(learning_rate=0.1,
                            max_depth = 6, 
                            n_estimators = 100,
                            scale_pos_weight=1.3)
model_imb_t3.fit(X_train, y_train)

XGBClassifier(max_depth=6, scale_pos_weight=1.3)

In [37]:
print("Accuracy on training set: {:.2f}".format(model_imb_t3.score(X_train, y_train) * 100))
print("Accuracy on validation set: {:.2f}".format(model_imb_t3.score(X_test, y_test) * 100))

Accuracy on training set: 85.89
Accuracy on validation set: 85.65


I don't think I will get better than this by decreasing even more the scale_pos_weight ,so I will take this as my best model taking into account the DEP_DELAY.

In [38]:
xgb_predict_m1 = model_imb_t3.predict(X_test)
xgb_predict_m1

array([1, 0, 0, ..., 0, 1, 1])

In [39]:
print(confusion_matrix(y_test, xgb_predict_m1))
print(classification_report(y_test, xgb_predict_m1))

[[1788881  100527]
 [ 330748  786036]]
              precision    recall  f1-score   support

           0       0.84      0.95      0.89   1889408
           1       0.89      0.70      0.78   1116784

    accuracy                           0.86   3006192
   macro avg       0.87      0.83      0.84   3006192
weighted avg       0.86      0.86      0.85   3006192

